In [12]:
import timeit
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
from tqdm import trange
import time

from bound_propagation import BoundModelFactory, HyperRectangle


class IBPNet(nn.Sequential):
    def __init__(self):
        super(IBPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [5]:
def construct_transform():
    transform = transforms.Compose([
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Lambda(torch.flatten)
    ])
    target_transform = transforms.Compose([])
    return transform, target_transform


def adversarial_logit(y_hat, y):
    """Compute adversarial logits by taking upper bounds for incorrect classes
    and lower bounds for the correct class."""
    batch_size = y.size(0)
    classes = torch.arange(10, device=y.device).unsqueeze(0).expand(batch_size, -1)
    mask = (classes == y.unsqueeze(-1)).to(dtype=y_hat.lower.dtype)
    
    # Take upper bound for logit of all but the correct class where you take the lower bound
    adversarial_logit = (1 - mask) * y_hat.upper + mask * y_hat.lower
    return adversarial_logit

In [14]:
def train_ibp(net, device, num_epochs=100, epsilon_target=0.1):
    """Train network with IBP robustness training following the paper's procedure.
    
    The training uses:
    - Curriculum learning with scheduled κ (1.0 → 0.5) and ε (0 → ε_target)
    - Combined loss: κ·CE(z_K, y) + (1-κ)·CE(ẑ_K(ε), y)
    - Linear warmup and ramp-up schedules as in paper Appendix A
    """
    print('[IBP TRAINING - Following Paper Protocol]')
    
    transform, target_transform = construct_transform()
    train_data = datasets.FashionMNIST('./fashion_data', train=True, download=True,
                                       transform=transform, target_transform=target_transform)
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True, num_workers=4)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    
    # Training schedule parameters (from paper Appendix A)
    total_steps = num_epochs * len(train_loader)
    warmup_steps = int(0.1 * total_steps)  # 10% warmup
    rampup_steps = int(0.5 * total_steps)  # 50% ramp-up for epsilon
    
    # Learning rate decay schedule
    lr_decay_steps = [int(0.6 * total_steps), int(0.9 * total_steps)]
    
    start_time = time.time()
    current_step = 0
    
    for epoch in trange(num_epochs):
        running_loss = 0.0
        running_ce = 0.0
        running_robust = 0.0
        
        for batch_idx, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            
            # Learning rate schedule
            if current_step == lr_decay_steps[0] or current_step == lr_decay_steps[1]:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.1
            
            # Linear warmup for learning rate
            if current_step < warmup_steps:
                lr_scale = current_step / warmup_steps
                for param_group in optimizer.param_groups:
                    param_group['lr'] = 1e-3 * lr_scale
            
            # Compute schedule values (linear ramp)
            # κ: from 1.0 to 0.5 over full training
            k = max(1.0 - 0.5 * (current_step / total_steps), 0.5)
            
            # ε: from 0 to ε_target over rampup period
            if current_step < rampup_steps:
                eps_train = epsilon_target * (current_step / rampup_steps)
            else:
                eps_train = epsilon_target
            
            optimizer.zero_grad(set_to_none=True)
            
            # Standard prediction and CE loss
            y_hat = net(X)
            ce_loss = criterion(y_hat, y)
            
            # IBP bound propagation and robustness loss
            bounds = net.ibp(HyperRectangle.from_eps(X, eps_train))
            adv_logit = adversarial_logit(bounds, y)
            robust_loss = criterion(adv_logit, y)
            
            # Combined loss (Eq. 12 from paper)
            loss = k * ce_loss + (1 - k) * robust_loss
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_ce += ce_loss.item()
            running_robust += robust_loss.item()
            current_step += 1
            
            if batch_idx % 50 == 49:
                print(f'Epoch [{epoch+1:3d}/{num_epochs}], Step [{batch_idx+1:3d}], '
                      f'Loss: {running_loss/50:.4f}, CE: {running_ce/50:.4f}, '
                      f'Robust: {running_robust/50:.4f}, ε: {eps_train:.4f}, κ: {k:.3f}')
                running_loss = 0.0
                running_ce = 0.0
                running_robust = 0.0
    
    training_time = time.time() - start_time
    print(f'IBP Training completed in {training_time:.2f} seconds')
    return training_time

In [15]:
def train_standard(net, device, num_epochs=100):
    """Standard training with cross-entropy loss for comparison."""
    print('[STANDARD TRAINING]')
    
    transform, target_transform = construct_transform()
    train_data = datasets.FashionMNIST('./fashion_data', train=True, download=True,
                                       transform=transform, target_transform=target_transform)
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True, num_workers=4)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    
    start_time = time.time()
    total_steps = num_epochs * len(train_loader)
    lr_decay_steps = [int(0.6 * total_steps), int(0.9 * total_steps)]
    current_step = 0
    
    for epoch in trange(num_epochs):
        running_loss = 0.0
        
        for batch_idx, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            
            # Learning rate decay schedule
            if current_step == lr_decay_steps[0] or current_step == lr_decay_steps[1]:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.1
            
            optimizer.zero_grad(set_to_none=True)
            
            y_hat = net(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            current_step += 1
            
            if batch_idx % 50 == 49:
                print(f'Epoch [{epoch+1:3d}/{num_epochs}], Step [{batch_idx+1:3d}], Loss: {running_loss/50:.4f}')
                running_loss = 0.0
    
    training_time = time.time() - start_time
    print(f'Standard Training completed in {training_time:.2f} seconds')
    return training_time

In [10]:
@torch.no_grad()
def test_standard_accuracy(net, device):
    """Evaluate standard accuracy on test set."""
    print('[STANDARD ACCURACY TEST]')
    
    transform, target_transform = construct_transform()
    test_data = datasets.FashionMNIST('./fashion_data', train=False, download=True,
                                      transform=transform, target_transform=target_transform)
    test_loader = DataLoader(test_data, batch_size=256, shuffle=False, num_workers=4)
    
    correct = 0
    total = 0
    
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        y_hat = net(X)
        predicted = torch.argmax(y_hat, dim=1)
        correct += (predicted == y).sum().item()
        total += y.size(0)
    
    accuracy = correct / total
    print(f'Standard Accuracy: {accuracy:.4f}')
    return accuracy



def test_robust_accuracy_pgd(net, device, epsilon=0.1, num_steps=20, step_size=None):
    """Evaluate robust accuracy against PGD attack."""
    print(f'[ROBUST ACCURACY TEST - PGD ε={epsilon}]')
    
    if step_size is None:
        step_size = epsilon / 4
    
    transform, target_transform = construct_transform()
    test_data = datasets.FashionMNIST('./fashion_data', train=False, download=True,
                                      transform=transform, target_transform=target_transform)
    test_loader = DataLoader(test_data, batch_size=256, shuffle=False, num_workers=4)
    
    correct = 0
    total = 0
    
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        
        # PGD attack
        X_adv = X.clone() + torch.empty_like(X).uniform_(-epsilon, epsilon)
        X_adv = torch.clamp(X_adv, 0, 1)
        X_adv.requires_grad = True
        
        for _ in range(num_steps):
            with torch.enable_grad():
                y_hat = net(X_adv)
                loss = F.cross_entropy(y_hat, y)
                grad = torch.autograd.grad(loss, X_adv)[0]
            
            with torch.no_grad():
                X_adv.data = X_adv + step_size * grad.sign()
                X_adv.data = torch.clamp(X_adv, X - epsilon, X + epsilon)
                X_adv.data = torch.clamp(X_adv, 0, 1)
            
            X_adv.requires_grad_(True)
        
        with torch.no_grad():
            y_hat = net(X_adv)
            predicted = torch.argmax(y_hat, dim=1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    
    accuracy = correct / total
    print(f'Robust Accuracy (PGD ε={epsilon}): {accuracy:.4f}')
    return accuracy



In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Train standard model
print('\n' + '='*60)
print('STANDARD TRAINING')
print('='*60)
net_standard = IBPNet().to(device)
std_train_time = train_standard(net_standard, device, num_epochs=20)
std_accuracy = test_standard_accuracy(net_standard, device)
std_robust_accuracy = test_robust_accuracy_pgd(net_standard, device, epsilon=0.1)

# Train IBP model
print('\n' + '='*60)
print('IBP ROBUST TRAINING')
print('='*60)
net_ibp = IBPNet().to(device)

# Build IBP-enabled model
factory = BoundModelFactory()
net_ibp = factory.build(net_ibp)

ibp_train_time = train_ibp(net_ibp, device, num_epochs=20, epsilon_target=0.1)
ibp_accuracy = test_standard_accuracy(net_ibp, device)
ibp_robust_accuracy = test_robust_accuracy_pgd(net_ibp, device, epsilon=0.1)

# Report results
print('\n' + '='*60)
print('RESULTS SUMMARY')
print('='*60)
print(f'\n{"Model":<15} {"Std Accuracy":<15} {"Rob Accuracy":<15} {"Train Time":<15}')
print('-'*60)
print(f'{"Standard":<15} {std_accuracy:<15.4f} {std_robust_accuracy:<15.4f} {std_train_time:<15.2f}s')
print(f'{"IBP Robust":<15} {ibp_accuracy:<15.4f} {ibp_robust_accuracy:<15.4f} {ibp_train_time:<15.2f}s')
print(f'\nRobustness Improvement: {(ibp_robust_accuracy - std_robust_accuracy)*100:.2f}%')
print(f'Training Time Ratio (IBP/Standard): {ibp_train_time/std_train_time:.2f}x')

Using device: cuda

STANDARD TRAINING
[STANDARD TRAINING]


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [  1/20], Step [ 50], Loss: 1.7397
Epoch [  1/20], Step [100], Loss: 0.8435
Epoch [  1/20], Step [150], Loss: 0.7270
Epoch [  1/20], Step [200], Loss: 0.6403
Epoch [  1/20], Step [250], Loss: 0.6333
Epoch [  1/20], Step [300], Loss: 0.5525
Epoch [  1/20], Step [350], Loss: 0.5439
Epoch [  1/20], Step [400], Loss: 0.5287
Epoch [  1/20], Step [450], Loss: 0.5286
Epoch [  1/20], Step [500], Loss: 0.5302
Epoch [  1/20], Step [550], Loss: 0.4902


  5%|▌         | 1/20 [00:03<01:04,  3.41s/it]

Epoch [  1/20], Step [600], Loss: 0.4894
Epoch [  2/20], Step [ 50], Loss: 0.4708
Epoch [  2/20], Step [100], Loss: 0.4765
Epoch [  2/20], Step [150], Loss: 0.4739
Epoch [  2/20], Step [200], Loss: 0.4496
Epoch [  2/20], Step [250], Loss: 0.4498
Epoch [  2/20], Step [300], Loss: 0.4389
Epoch [  2/20], Step [350], Loss: 0.4293
Epoch [  2/20], Step [400], Loss: 0.4422
Epoch [  2/20], Step [450], Loss: 0.4223
Epoch [  2/20], Step [500], Loss: 0.4306
Epoch [  2/20], Step [550], Loss: 0.4100


...

 95%|█████████▌| 19/20 [05:47<00:18, 18.01s/it]

Epoch [ 19/20], Step [600], Loss: 1.2750, CE: 0.7148, Robust: 1.8968, ε: 0.1000, κ: 0.525
Epoch [ 20/20], Step [ 50], Loss: 1.2495, CE: 0.6954, Robust: 1.8595, ε: 0.1000, κ: 0.523
Epoch [ 20/20], Step [100], Loss: 1.2502, CE: 0.6852, Robust: 1.8670, ε: 0.1000, κ: 0.521
Epoch [ 20/20], Step [150], Loss: 1.2658, CE: 0.7064, Robust: 1.8714, ε: 0.1000, κ: 0.519
Epoch [ 20/20], Step [200], Loss: 1.2329, CE: 0.6809, Robust: 1.8254, ε: 0.1000, κ: 0.517
Epoch [ 20/20], Step [250], Loss: 1.2680, CE: 0.7093, Robust: 1.8628, ε: 0.1000, κ: 0.515
Epoch [ 20/20], Step [300], Loss: 1.2345, CE: 0.6779, Robust: 1.8222, ε: 0.1000, κ: 0.513
Epoch [ 20/20], Step [350], Loss: 1.2633, CE: 0.6890, Robust: 1.8645, ε: 0.1000, κ: 0.510
Epoch [ 20/20], Step [400], Loss: 1.2732, CE: 0.7036, Robust: 1.8646, ε: 0.1000, κ: 0.508
Epoch [ 20/20], Step [450], Loss: 1.2728, CE: 0.7051, Robust: 1.8573, ε: 0.1000, κ: 0.506
Epoch [ 20/20], Step [500], Loss: 1.2719, CE: 0.6972, Robust: 1.8587, ε: 0.1000, κ: 0.504
Epoch [ 20

100%|██████████| 20/20 [06:07<00:00, 18.37s/it]

Epoch [ 20/20], Step [600], Loss: 1.2296, CE: 0.6696, Robust: 1.7920, ε: 0.1000, κ: 0.500
IBP Training completed in 367.37 seconds
[STANDARD ACCURACY TEST]


Standard Accuracy: 0.7728
[ROBUST ACCURACY TEST - PGD ε=0.1]
Robust Accuracy (PGD ε=0.1): 0.5037

RESULTS SUMMARY

Model           Std Accuracy    Rob Accuracy    Train Time     
------------------------------------------------------------
Standard        0.8834          0.0049          66.66          s
IBP Robust      0.7728          0.5037          367.37         s

Robustness Improvement: 49.88%
Training Time Ratio (IBP/Standard): 5.51x


In [17]:
from tqdm import trange, tqdm
def verify_with_boundprop(bound_model, test_loader, eps_list, device):
    """Verify robustness using interval bound propagation.
    
    Following the example code approach with verification condition:
    lower_bound[true_class] > max(upper_bound[other_classes])
    """
    bound_model.eval()
    verified_accs = []
    results = {}
    
    for eps in eps_list:
        total = 0
        verified = 0
        
        for images, labels in tqdm(test_loader, desc=f'Verifying ε={eps:.3f}'):
            images, labels = images.to(device), labels.to(device)
            images = images.view(images.size(0), -1)  # Flatten manually
            
            # Create L∞ bounds
            input_bounds = HyperRectangle.from_eps(images, eps)
            
            # Interval Bound Propagation
            bounds = bound_model.ibp(input_bounds)
            lower, upper = bounds.lower, bounds.upper
            
            # Verification condition: l_i[true] > max(u_i[j] for j != true)
            for i in range(images.size(0)):
                true = labels[i].item()
                l_i = lower[i]
                u_i = upper[i]
                
                # Get max upper bound for incorrect classes
                other_classes = torch.arange(10, device=device) != true
                max_other_upper = u_i[other_classes].max()
                
                if l_i[true] > max_other_upper:
                    verified += 1
                total += 1
        
        acc = 100.0 * verified / total
        verified_accs.append(acc)
        results[eps] = {'verified': verified, 'total': total, 'acc': acc}
        print(f"ε={eps:.3f} → Verified accuracy: {acc:.2f}% ({verified}/{total})")
    
    return verified_accs, results

In [19]:
# Compute verified accuracy
import numpy as np
print('\n' + '='*70)
print('VERIFIED ACCURACY EVALUATION (Box Verification with IBP)')
print('='*70)

eps_list = np.linspace(0.01, 0.1, 10)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}\n')

# Load test data for verification
transform, target_transform = construct_transform()
test_data = datasets.FashionMNIST('./fashion_data', train=False, download=True,
                                    transform=transform, target_transform=target_transform)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4)
verified_accs, results = verify_with_boundprop(net_ibp, test_loader, eps_list, device)


VERIFIED ACCURACY EVALUATION (Box Verification with IBP)
Using device: cuda



Verifying ε=0.010:   0%|          | 0/157 [00:00<?, ?it/s]

Verifying ε=0.010: 100%|██████████| 157/157 [00:33<00:00,  4.70it/s]


ε=0.010 → Verified accuracy: 74.12% (7412/10000)


Verifying ε=0.020: 100%|██████████| 157/157 [00:14<00:00, 10.95it/s]


...

ε=0.080 → Verified accuracy: 47.72% (4772/10000)


Verifying ε=0.090: 100%|██████████| 157/157 [00:37<00:00,  4.14it/s]


ε=0.090 → Verified accuracy: 42.83% (4283/10000)


Verifying ε=0.100: 100%|██████████| 157/157 [00:26<00:00,  5.97it/s]

ε=0.100 → Verified accuracy: 38.38% (3838/10000)
